This notebook helps to have a first look at the saved output. 

# Select event type and parameter

In [ ]:
# ------------------------------------------------------------------------------------------ which data do you want to read?
run = "r001" # 000, 001, ...
subsystem = "geds" # KEEP 'geds' for the moment
folder = "prod-ref" # you can change me
period = "p03"
version = "" # leave an empty string if you're looking at p03 data

if version == "":
    data_file = f"/data1/users/calgaro/{folder}/generated/plt/phy/{period}/{run}/l200-{period}-{run}-phy-{subsystem}"
else:
    data_file = f"/data1/users/calgaro/{folder}/{version}/generated/plt/phy/{period}/{run}/l200-{period}-{run}-phy-{subsystem}"

import sys
import shelve
import matplotlib
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt
from legend_data_monitor import plot_styles, plotting
%matplotlib widget

# ------------------------------------------------------------------------------------------ load data
# Load the shelve object
shelf = shelve.open(data_file)

# ------------------------------------------------------------------------------------------ evt type
# Get the list of available event types
event_types = list(shelf["monitoring"].keys())

# Create a dropdown widget for the event type
evt_type_widget = widgets.Dropdown(options=event_types, description="Event Type:")


# ------------------------------------------------------------------------------------------ parameter
# Define a function to update the parameter dropdown based on the selected event type
def update_params(*args):
    selected_evt_type = evt_type_widget.value
    params = list(shelf["monitoring"][selected_evt_type].keys())
    param_widget.options = params


# Call the update_params function when the event type is changed
evt_type_widget.observe(update_params, "value")

# Create a dropdown widget for the parameter
param_widget = widgets.Dropdown(description="Parameter:")

# ------------------------------------------------------------------------------------------ data format
data_format = ['absolute values', '% values']

# Create a dropdown widget 
data_format_widget = widgets.Dropdown(
    options=data_format,
    description='data format:'
)

# ------------------------------------------------------------------------------------------ plot structure
plot_structures = ["per string", "per channel"]

# Create a dropdown widget
plot_structures_widget = widgets.Dropdown(
    options=plot_structures, description="Plot structure:"
)

# ------------------------------------------------------------------------------------------ plot style
plot_styles = ["vs time", "histogram"]

# Create a dropdown widget
plot_styles_widget = widgets.Dropdown(options=plot_styles, description="Plot style:")

# ------------------------------------------------------------------------------------------ resampling
# resampled='only' means you look at each timestamp entry mediated over 1H time window (to reduce the number of points in plots)
# resampled='no' means you look at each timestamp entry
# resampled='also' means you look at each timestamp entry mediated over 1H time window AND at each timestamp entry TOGETHER
#     \__suggestion: use 'also' just when you choose plot_structures='per channel';
#                    if you have selected 'per string', then you're not going to understand anything
resampled = ["no", "only", "yes"]

# Create a dropdown widget
resampled_widget = widgets.Dropdown(options=resampled, description="Resampled:")


# ------------------------------------------------------------------------------------------ get one or all strings
strings = [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, "all"]

# Create a dropdown widget
strings_widget = widgets.Dropdown(options=strings, description="String:")

# ------------------------------------------------------------------------------------------ display widgets
display(evt_type_widget)
display(
    param_widget
)  # it takes a while before displaying available parameters in the corresponding widget

# ------------------------------------------------------------------------------------------ get params (based on event type)
evt_type = evt_type_widget.value
params = list(shelf["monitoring"][evt_type].keys())
param_widget.options = params

print("If you change me, then RUN AGAIN the next cell!!!")

In [ ]:
# ------------------------------------------------------------------------------------------ get dataframe
def display_param_value(*args):
    selected_evt_type = evt_type_widget.value
    selected_param = param_widget.value
    print(
        f"You are going to plot '{selected_param}' for '{selected_evt_type}' events..."
    )
    # get dataframe
    df_param = shelf["monitoring"][selected_evt_type][selected_param]["df_geds"]
    # get plot info
    plot_info = shelf["monitoring"][selected_evt_type][selected_param]["plot_info"]

    return df_param, plot_info


df_param, plot_info = display_param_value()
print(f"...data have beeng loaded!")

# Plot data (select style and string)

In [ ]:
# ------------------------------------------------------------------------------------------ get plots
display(data_format_widget)
display(plot_structures_widget)
display(plot_styles_widget)
display(resampled_widget)
display(strings_widget)
print("If you change me, then RUN AGAIN the next cell!!!")

In [ ]:
# set plotting options
plot_info["plot_style"] = plot_styles_widget.value
plot_info["resampled"] = resampled_widget.value
plot_info["title"] = ""  # for plotting purposes
plot_info["subsystem"] = ""  # for plotting purposes

if data_format_widget.value == "absolute values":
    plot_info["parameter"] = plot_info['parameter'].split("_var")[0] if "_var" in plot_info["parameter"] else plot_info["parameter"]
    plot_info["unit_label"] = plot_info["unit"]
    if plot_info["parameter"] not in df_param:
        print("There is no", plot_info["parameter"])
        sys.exit("Stopping notebook.")
if data_format_widget.value == "% values":
    plot_info["parameter"] = plot_info['parameter'] if "_var" in plot_info["parameter"] else plot_info["parameter"] + "_var"
    plot_info["unit_label"] = "%"
    if plot_info["parameter"] not in df_param:
        print("There is no", plot_info["parameter"])
        sys.exit("Stopping notebook.")

print(f"Making plots now...")
if isinstance(strings_widget.value, str):  # let's get all strings in output
    for string in [1, 2, 3, 4, 5, 7, 8, 9, 10, 11]:
        if plot_structures_widget.value == "per channel":
            plotting.plot_per_ch(
                df_param[df_param["location"] == string], plot_info, ""
            )  # plot one canvas per channel
        elif plot_structures_widget.value == "per string":
            plotting.plot_per_string(
                df_param[df_param["location"] == string], plot_info, ""
            )  # plot one canvas per string
else:  # let's get one string in output
    if plot_structures_widget.value == "per channel":
        plotting.plot_per_ch(
            df_param[df_param["location"] == strings_widget.value], plot_info, ""
        )  # plot one canvas per channel
    elif plot_structures_widget.value == "per string":
        plotting.plot_per_string(
            df_param[df_param["location"] == strings_widget.value], plot_info, ""
        )  # plot one canvas per string

# Plot means over channels

In [ ]:
# ------------------------------------------------------------------------------------------ get means plot
# here you can monitor how the mean value (evaluated over the first 10% of data) behaves separately for different channels;
# these mean values were used to compute the % variations in 'vs time' plots.
# The legend of plot below shows the average of means of different channels belonging to the same string
plot_info["plot_style"] = "vs ch"
plot_info["unit_label"] = plot_info["unit"]
plot_info["parameter"] = plot_info['parameter'].split("_var")[0] if "_var" in plot_info["parameter"] else plot_info["parameter"]
plot_info["unit_label"] = plot_info["unit"]
data = df_param.drop(columns=[param_widget.value, "cc4_id", "HV_channel", "HV_card", "cc4_channel", "daq_crate", "det_type", "status", "timestamp"])
data = data.rename(columns={param_widget.value + "_mean": param_widget.value})
plotting.plot_array(data, plot_info, "")